In [2]:

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem.wordnet import WordNetLemmatizer
import elasticsearch
import pandas as pd





import os
import glob
os.chdir("C:/Users/vishw/Downloads/articles/articles")
files = glob.glob("*.*")
print(len(files))





def nlpPipeline(art):
    tokens = nltk.word_tokenize(art)
    lemmatizer=WordNetLemmatizer();
    for i in range(len(tokens)):
        tokens[i] = lemmatizer.lemmatize(tokens[i])
    return (nltk.pos_tag(tokens)) 
    





#print(files[0].read())




def read_text_file(file_path):
    with open(file_path, 'r',encoding = 'utf8') as f:
        return f.read()




content = read_text_file(files[0])

















30
[('Bird', 'NNP'), ('migration', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('regular', 'JJ'), ('seasonal', 'JJ'), ('movement', 'NN'), (',', ','), ('often', 'RB'), ('north', 'JJ'), ('and', 'CC'), ('south', 'JJ'), ('along', 'IN'), ('a', 'DT'), ('flyway', 'NN'), (',', ','), ('between', 'IN'), ('breeding', 'VBG'), ('and', 'CC'), ('wintering', 'VBG'), ('ground', 'NN'), ('.', '.'), ('Many', 'JJ'), ('specie', 'NN'), ('of', 'IN'), ('bird', 'NN'), ('migrate', 'NN'), ('.', '.'), ('Migration', 'NNP'), ('carry', 'VBP'), ('high', 'JJ'), ('cost', 'NN'), ('in', 'IN'), ('predation', 'NN'), ('and', 'CC'), ('mortality', 'NN'), (',', ','), ('including', 'VBG'), ('from', 'IN'), ('hunting', 'VBG'), ('by', 'IN'), ('human', 'JJ'), (',', ','), ('and', 'CC'), ('is', 'VBZ'), ('driven', 'VBN'), ('primarily', 'RB'), ('by', 'IN'), ('availability', 'NN'), ('of', 'IN'), ('food', 'NN'), ('.', '.'), ('It', 'PRP'), ('occurs', 'VBZ'), ('mainly', 'RB'), ('in', 'IN'), ('the', 'DT'), ('northern', 'NN'), ('hemisphere', 'RB'), 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vishw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vishw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vishw\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:

print(elasticsearch.VERSION)




df = pd.DataFrame(columns = ("name","content" ))



dict1={}
this_loc = 0
for file in files:
    content = read_text_file(file)
    sent_text = nltk.sent_tokenize(content) 
    for i in sent_text:
        df.loc[this_loc] = file,i
        dict1[file]=i
        this_loc += 1
    
    










(7, 15, 2)


In [81]:
df.tail()

,name,content
7115,6.txt,"In this meeting with the Qing emperor, Goldste..."
7116,6.txt,Van Praag states that Tibet and the Dalai Lama...
7117,6.txt,When the Dzungar Mongols attempted to spread t...
7118,6.txt,"By 1751, during the reign of the Qianlong Empe..."
7119,6.txt,"As of 1751, Albert Kolb writes that ""Chinese c..."


In [82]:
from elasticsearch import Elasticsearch
print(elasticsearch.VERSION)


(7, 15, 2)


In [83]:
es=Elasticsearch('http://localhost:9200/')

In [84]:
col_names=df.columns
for row_number in range(df.shape[0]):
    body=dict([(name, str(df.iloc[row_number][name])) for name in col_names])
    es.index(index='nlp_content',doc_type='books',document=body)

In [143]:
input=" What is ukiyo-e an example of?"
question_type=""
look_for=""
if " What " in input:
    question_type="what"
    look_for="ORGANIZATION"
    
if " who " or " whom " in input:
    question_type="who"
    look_for="PERSON"
    
if " when " in input:
    question_type="when"
    look_for="DATE"

import spacy
import en_core_web_md
spacy_model = en_core_web_md.load()
print(look_for)

question=nlpPipeline(input)

PERSON


In [138]:
search_results=es.search(index='nlp_content',doc_type='books',
                        body={"_source":"content","from" : 0, "size" : 100,
                             'query':{
                                 'match':{"content":input }, 
                             }
                             })

C:\Users\vishw\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  after removing the cwd from sys.path.


In [141]:
my_set=set()

my_list=[]
for i in range(0,100):
    my_set.add(search_results['hits']['hits'][i]['_source']['content'])
    my_list.append(search_results['hits']['hits'][i]['_source']['content'])
#print(my_set)

Answer_sent=""
max_score=0;
flag=False

#pip install spacy
import spacy as sp
nlp = spacy.load("en_core_web_md")

for i in my_set:
    flag= False
    entity_doc = spacy_model(i)
    
        
    for entity in entity_doc.ents:
        #print(entity.text, entity .label_ , look_for)
        if entity.label_ is look_for:
            flag=True
            
    if flag is True:
        input=nlp(input)
        i=nlp(i)
        score=i.similarity(input)
        if(score>max_score):
            Answer_sent=i
            max_score=score
        
        
    
    
    
    
print(Answer_sent)
print(max_score)


    

[dubious – discuss] Mass culture refers to the mass-produced and mass mediated forms of consumer culture that emerged in the 20th century.
0.8682030030881775


In [116]:
search_results

{'took': 212,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3090, 'relation': 'eq'},
  'max_score': 15.1411,
  'hits': [{'_index': 'nlp_content',
    '_type': 'books',
    '_id': 'Gq1-K30BdN57Yi-cIc2t',
    '_score': 15.1411,
    '_source': {'content': 'mass culture).'}},
   {'_index': 'nlp_content',
    '_type': 'books',
    '_id': '6q1PLH0BdN57Yi-ctuj6',
    '_score': 15.1411,
    '_source': {'content': 'mass culture).'}},
   {'_index': 'nlp_content',
    '_type': 'books',
    '_id': 'uq5WLH0BdN57Yi-coAT3',
    '_score': 15.1411,
    '_source': {'content': 'mass culture).'}},
   {'_index': 'nlp_content',
    '_type': 'books',
    '_id': 'iq5pLH0BdN57Yi-cciCT',
    '_score': 15.1411,
    '_source': {'content': 'mass culture).'}},
   {'_index': 'nlp_content',
    '_type': 'books',
    '_id': 'oyJbNX0BjpmUqUpewexj',
    '_score': 15.1411,
    '_source': {'content': 'mass culture).'}},
   {'_index': 'nlp_content',
